In [1]:
# Все библиотеки

import pandas as pd
from scipy.stats import ttest_ind, levene, mannwhitneyu,shapiro, \
    f_oneway, kruskal, chi2_contingency, fisher_exact
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve
import pingouin as pg
import warnings

warnings.filterwarnings('ignore')

In [2]:
income = pd.read_csv('заработок бобров.csv', encoding='utf-8', sep=';')
purs = pd.read_csv('покупки.csv', encoding='utf-8', sep=';')
staff = pd.read_excel('сотрудники.xlsx')

In [3]:
def obr(df):
    df.columns = df.columns.str.replace(' ', '_').str.lower().str.replace('.', '')
    return df

In [4]:
income = obr(income)
purs = obr(purs)
staff = obr(staff)

In [5]:
missions_to_drop = [
    'тренер',
    'Croc sport',
    'Croc sport( 4 уровень)',
    'миссия best hunter',
    'миссия креативный класс',
    'путешественник КРОК',
    'развитие бренда работодателя',
    'продвижение IT-профессий'
]

comments_to_drop = ['ошибочно начислены']

income = income[~income['название_миссии'].str.lower().isin([m.lower() for m in missions_to_drop])]
income = income[~income['комментарий'].str.lower().isin([m.lower() for m in comments_to_drop])]

In [6]:
income['дата'] = pd.to_datetime(income['дата'], format='%d.%m.%Y')
today = pd.Timestamp.today().normalize()
income['дней_назад'] = (today - income['дата']).dt.days

In [7]:
purs['дата_оформления_заказа'] = pd.to_datetime(purs['дата_оформления_заказа'], format='%d.%m.%Y %H:%M')
today = pd.Timestamp.today().normalize()
purs['дней_назад'] = (today - purs['дата_оформления_заказа'].dt.normalize()).dt.days

In [8]:
import re


def parse_stazh_to_days(text):
    years = months = days = 0

    year_match = re.search(r'(\d+)\s*г\.', text)
    if year_match:
        years = int(year_match.group(1))

    month_match = re.search(r'(\d+)\s*мес\.', text)
    if month_match:
        months = int(month_match.group(1))

    day_match = re.search(r'(\d+)\s*дн\.', text)
    if day_match:
        days = int(day_match.group(1))

    total_days = years * 360 + months * 30 + days
    return total_days

staff['стаж_в_днях'] = staff['стаж_фактический_по_компании'].apply(parse_stazh_to_days)

In [9]:
staff['код_сотрудника'] = staff['внешний_код']
purs = purs.merge(
    staff[['код_сотрудника', 'стаж_в_днях']],
    on='код_сотрудника',
    how='left'
)
purs['стаж_на_момент_покупки'] = purs['стаж_в_днях'] - purs['дней_назад']


income = income.merge(
    staff[['код_сотрудника', 'стаж_в_днях']],
    on='код_сотрудника',
    how='left'
)
income['стаж_на_момент_заработка'] = income['стаж_в_днях'] - income['дней_назад']

In [10]:
employees_with_missions = income['код_сотрудника'].unique()
employees_with_purchases = purs['код_сотрудника'].unique()

all_employees = set(employees_with_missions) | set(employees_with_purchases)
percent_with_missions = len(employees_with_missions) / len(all_employees) * 100
percent_with_purchases = len(employees_with_purchases) / len(all_employees) * 100

print(f"Процент людей с хотя бы одной миссией: {percent_with_missions:.2f}%")
print(f"Процент людей с хотя бы одной покупкой: {percent_with_purchases:.2f}%")

Процент людей с хотя бы одной миссией: 73.90%
Процент людей с хотя бы одной покупкой: 92.61%


In [11]:
purs['сумма'] = purs['стоимость_в_валюте'] * purs['количество'] * (1 - purs['скидка']/100)
total_spent = purs['сумма'].sum()
unique_buyers = purs['код_сотрудника'].nunique()
avg_spent_per_person = total_spent / unique_buyers

print(f"Средняя сумма, потраченная одним сотрудником: {avg_spent_per_person:.2f}")

Средняя сумма, потраченная одним сотрудником: 8116.20


In [12]:
good = [
'Ты вовремя списал затраченное время на задачи и проекты',
'Ты вовремя подписал документы по кадровым движениям',
'Креативный класс',
'Ты проверил информацию при обновлении проектного опыта (для юристов, за 1 проект)',
'Ты прошел все онлайн-курсы для новых сотрудников',
'Ты проверил информацию при сборе проектного опыта (для маркетологов, специалистов службы качества, за 1 проект)',
'Ты заполнил проектный опыт (для менеджеров проекта, за 1 проект)',
'Ты повторно проверил информацию при обновлении проектного опыта (для юристов, за 1 проект)',
'Ты повторно проверил информацию при обновлении проектного опыта (для маркетологов, специалистов службы качества, за 1 проект)',
'Миссия Креативный класс',
'Ты обновил полностью проектный опыт (для в менеджера проекта, за 1 проект)',
'Ты успешно сдал экзамен*',
'Ты прошел программу адаптации для аналитиков',
'Ты сдал экзамен* на 100%'
]

In [13]:
from scipy.stats import wilcoxon


def good_count(missions):
    return sum(mission in good for mission in missions)


def effectivnost(missions):
    total = len(missions)
    if total == 0:
        return 0.0
    return good_count(missions) / total


def sravnenie(df, missions):
    df['дата'] = pd.to_datetime(df['дата'])

    results = []
    for employee_id, group in df[['код_сотрудника', 'дата', 'название_миссии']].groupby('код_сотрудника'):
        group = group.sort_values('дата')
        top = group[group['название_миссии'].str.contains('|'.join(missions), na=False)]
        if len(top) == 0:
            continue

        if not top.empty:
            top_data = top['дата'].iloc[0]
        else:
            top_data = pd.Timestamp.max

        missions_before = group[group['дата'] < top_data]['название_миссии']
        missions_after = group[group['дата'] >= top_data]['название_миссии']

        start_date = pd.to_datetime("2023-09-02")
        before_diff = (top_data - start_date).days

        end_date = pd.to_datetime("2024-10-15")
        after_diff = (end_date - top_data).days

        k_before = len(missions_before)/before_diff
        k_after = len(missions_after)/after_diff

        good_k_before = good_count(missions_before)/before_diff
        good_k_after = good_count(missions_after)/after_diff

        eff_before = effectivnost(missions_before)
        eff_after = effectivnost(missions_after)

        results.append({
            'код_сотрудника': employee_id,
            'эффективность_до': eff_before,
            'эффективность_после': eff_after,
            'k_before': k_before,
            'k_after': k_after,
            'good_k_before': good_k_before,
            'good_k_after': good_k_after,
            'missions_before': missions_before.shape[0],
            'missions_after': missions_after.shape[0]
        })

    eff_df = pd.DataFrame(results)

    valid_eff = eff_df.dropna(subset=['эффективность_до', 'эффективность_после'])

    stat, p_value = wilcoxon(valid_eff['эффективность_до'], valid_eff['эффективность_после'])
    stat2, p_value2 = wilcoxon(valid_eff['k_before'], valid_eff['k_after'])
    stat3, p_value3 = wilcoxon(valid_eff['good_k_before'], valid_eff['good_k_after'])

    print(f"{missions} :\nПРОЦЕНТЫ\nWilcoxon test: статистика = {stat}, p-value = {p_value:.5f}")

    if p_value < 0.05:
        print("📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)")
    else:
        print("📉 Статистически значимой разницы не найдено (p ≥ 0.05)")


    print()
    print(f"до : {eff_df['эффективность_до'].median()}")
    print(f"после : {eff_df['эффективность_после'].median()}")
    print()

    print(f"КОЛ-ВО ЗАДАЧ В ДЕНЬ\nWilcoxon test: статистика = {stat2}, p-value = {p_value2:.5f}")

    if p_value2 < 0.05:
        print("📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)")
    else:
        print("📉 Статистически значимой разницы не найдено (p ≥ 0.05)")

    print()
    print(f"до : {eff_df['k_before'].median()}")
    print(f"после : {eff_df['k_after'].median()}")
    print()

    print(f"КОЛ-ВО КРУТЫХ ЗАДАЧ В ДЕНЬ\nWilcoxon test: статистика = {stat3}, p-value = {p_value3:.5f}")

    if p_value3 < 0.05:
        print("Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)")
    else:
        print("Статистически значимой разницы не найдено (p ≥ 0.05)")

    print()
    print(f"до : {eff_df['good_k_before'].median()}")
    print(f"после : {eff_df['good_k_after'].median()}")

    print()
    print(f"медианное кол-во миссий до: {eff_df['missions_before'].median()}")
    print(f"медианное кол-во миссий после: {eff_df['missions_after'].median()}")


TheBest = ['HiPro', 'HiPo', 'Креативный класс', 'Звезда департамента']
sravnenie(income, TheBest)

['HiPro', 'HiPo', 'Креативный класс', 'Звезда департамента'] :
ПРОЦЕНТЫ
Wilcoxon test: статистика = 247780.5, p-value = 0.00048
📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.2
после : 0.3333333333333333

КОЛ-ВО ЗАДАЧ В ДЕНЬ
Wilcoxon test: статистика = 26403.0, p-value = 0.00000
📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.024096385542168676
после : 0.07920792079207921

КОЛ-ВО КРУТЫХ ЗАДАЧ В ДЕНЬ
Wilcoxon test: статистика = 47683.0, p-value = 0.00000
Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.005780346820809248
после : 0.021739130434782608

медианное кол-во миссий до: 5.0
медианное кол-во миссий после: 11.0


In [14]:
from scipy.stats import wilcoxon

def sravnenie_do_i_posle(income, purs, the_best):
    income['дата'] = pd.to_datetime(income['дата'])
    purs['дата_оформления_заказа'] = pd.to_datetime(purs['дата_оформления_заказа'])

    result = []

    for employee_id, group in income.groupby('код_сотрудника'):
        group = group.sort_values('дата')
        top_missions = group[group['название_миссии'].str.contains('|'.join(the_best), na=False)]
        if top_missions.empty:
            continue

        top_date = top_missions['дата'].iloc[0]

        before_missions = group[(group['дата'] < top_date) & ~group['название_миссии'].isin(the_best)]
        after_missions = group[(group['дата'] >= top_date) & ~group['название_миссии'].isin(the_best)]

        def count_purchases(mission_dates):
            count = 0
            total = len(mission_dates)
            for date in mission_dates:
                purchases = purs[
                    (purs['код_сотрудника'] == employee_id) &
                    (purs['дата_оформления_заказа'] > date) &
                    (purs['дата_оформления_заказа'] <= date + pd.Timedelta(days=7))
                ]
                if not purchases.empty:
                    count += 1
            return count, total

        before_count, before_total = count_purchases(before_missions['дата'])
        after_count, after_total = count_purchases(after_missions['дата'])

        if before_total > 0 and after_total > 0:
            result.append({
                'код_сотрудника': employee_id,
                'доля_до': before_count / before_total,
                'доля_после': after_count / after_total
            })

    result_df = pd.DataFrame(result)
    stat, p_value = wilcoxon(result_df['доля_до'], result_df['доля_после'])

    print("Wilcoxon test по долям миссий с покупками:")
    print(f"статистика = {stat}, p-value = {p_value:.5f}")
    if p_value < 0.05:
        print("Есть статистически значимая разница (p < 0.05)")
    else:
        print("Нет статистически значимой разницы (p ≥ 0.05)")

    print()
    print(f"Медианная доля покупок ДО: {result_df['доля_до'].median():.3f}")
    print(f"Медианная доля покупок ПОСЛЕ: {result_df['доля_после'].median():.3f}")


sravnenie_do_i_posle(income, purs, TheBest)

Wilcoxon test по долям миссий с покупками:
статистика = 80056.5, p-value = 0.00000
Есть статистически значимая разница (p < 0.05)

Медианная доля покупок ДО: 0.000
Медианная доля покупок ПОСЛЕ: 0.083


In [15]:
for i in staff['является_рм/тл'].value_counts().head(3).reset_index()['является_рм/тл'].unique():
    print(i)
    sravnenie(income[income['код_сотрудника'].isin(staff[staff['является_рм/тл'] == i]['код_сотрудника'].tolist())], TheBest)

нет
['HiPro', 'HiPo', 'Креативный класс', 'Звезда департамента'] :
ПРОЦЕНТЫ
Wilcoxon test: статистика = 167023.0, p-value = 0.01303
📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.23904761904761904
после : 0.3333333333333333

КОЛ-ВО ЗАДАЧ В ДЕНЬ
Wilcoxon test: статистика = 16941.5, p-value = 0.00000
📈 Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.027399111217871724
после : 0.0850531914893617

КОЛ-ВО КРУТЫХ ЗАДАЧ В ДЕНЬ
Wilcoxon test: статистика = 32014.0, p-value = 0.00000
Есть статистически значимая разница между эффективностью ДО и ПОСЛЕ ключевых значений (p < 0.05)

до : 0.005917159763313609
после : 0.025532377227292482

медианное кол-во миссий до: 6.0
медианное кол-во миссий после: 11.0
РМ
['HiPro', 'HiPo', 'Креативный класс', 'Звезда департамента'] :
ПРОЦЕНТЫ
Wilcoxon test: статистика = 1695.0, p-value = 0.00963
📈 Есть статистически значимая разница между эффективн

In [24]:
def avg_top_date(df, missions):
    df['дата'] = pd.to_datetime(df['дата'])

    top_dates = []
    for employee_id, group in df[['код_сотрудника', 'дата', 'название_миссии']].groupby('код_сотрудника'):
        group = group.sort_values('дата')
        top = group[group['название_миссии'].str.contains('|'.join(missions), na=False)]
        if not top.empty:
            top_dates.append(top['дата'].iloc[0])

    avg_date = pd.to_datetime(top_dates).mean()
    print(f"📅 Средняя дата получения первой миссии из TheBest: {avg_date.date()}")
    return avg_date


avg_top_date(income, TheBest)


📅 Средняя дата получения первой миссии из TheBest: 2024-04-24


Timestamp('2024-04-24 13:42:41.702127616')

In [23]:
from scipy.stats import wilcoxon

def temporal_sravnenie(df):
    df['дата'] = pd.to_datetime(df['дата'])
    median_date = pd.to_datetime('2024-04-24')

    results = []
    for employee_id, group in df[['код_сотрудника', 'дата', 'название_миссии']].groupby('код_сотрудника'):
        group = group.sort_values('дата')

        before = group[group['дата'] < median_date]
        after = group[group['дата'] >= median_date]

        before_days = (median_date - before['дата'].min()).days if not before.empty else 1
        after_days = (after['дата'].max() - median_date).days if not after.empty else 1

        good_k_before = good_count(before['название_миссии']) / before_days if before_days > 0 else 0
        good_k_after = good_count(after['название_миссии']) / after_days if after_days > 0 else 0

        results.append({
            'код_сотрудника': employee_id,
            'good_k_before': good_k_before,
            'good_k_after': good_k_after
        })

    temp_df = pd.DataFrame(results).dropna()

    stat, p_value = wilcoxon(temp_df['good_k_before'], temp_df['good_k_after'])

    print("🔎 Проверка общей временной тенденции (до / после медианной даты):")
    print(f"Wilcoxon test: статистика = {stat}, p-value = {p_value:.5f}")
    if p_value < 0.05:
        print("Есть статистически значимая разница между ДО и ПОСЛЕ по времени")
    else:
        print("Разницы по времени не обнаружено")

    print()
    print(f"до : {temp_df['good_k_before'].median()}")
    print(f"после : {temp_df['good_k_after'].median()}")

temporal_sravnenie(income)


🔎 Проверка общей временной тенденции (до / после медианной даты):
Wilcoxon test: статистика = 1540260.0, p-value = 0.00000
📈 Есть статистически значимая разница между ДО и ПОСЛЕ по времени

до : 0.010101010101010102
после : 0.012461180124223602
